In [17]:
import unicodecsv as csv
import urllib.parse
import urllib.request
import time
import re
from itertools import islice
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from urllib.request import Request, urlopen

In [18]:
#Reliable sources
fpath_nyt = "nyt.txt"
fpath_wsj = "wsj.txt"
fpath_bbc = "bbc.txt"

#Unreliable sources
#Need more
fpath_bb = "bb.txt"

#ON HOLD, POTENTIALLY A STRETCH GOAL
#Satire sources
#fpath_onion = "theonionlinks.txt"
tp = {"wsj", "nyt", "bbc", "bb", "an"}
articles = {"wsj": ['wsj.txt', 'wsj.csv'], "nyt": ['nyt.txt', 'nyt.csv'],
            "bbc": ['bbc.txt', 'bbc.csv'], "bb": ['bb.txt', 'bb.csv'],
           "an": ['an.txt', 'an.csv']}

In [30]:
def Scrape(name):
    with open(articles[name][1], 'wb') as result:
        if name == "wsj":
            writer = csv.writer(result, dialect='excel')
            driver = webdriver.Firefox()
            with open(articles[name][0], 'r', encoding="utf-8") as f:
                for url in f:
                    encoded = urllib.parse.quote(url.encode("utf-8"))
                    fullEncode = "https://m.facebook.com/l.php?u=" + encoded
            
                    driver.get(fullEncode)
                    driver.find_element_by_xpath("//a[@class='_42ft _42fu selected _42g-']").click()
                    time.sleep(20)
                    driver.find_element_by_xpath("//div[@class='close-btn']").click()
                    source = driver.page_source
                    soup = BeautifulSoup(source, 'lxml')
                    story = soup.findAll('p')
                    st = ""
                    for x in story:
                        st = st + str(x.get_text().replace('\n', '').replace('\r', ''))
                    fin = [st]
                    writer.writerow(fin)
            #driver.close()
        else:
            writer = csv.writer(result, dialect='excel')
            with open(articles[name][0], 'r', encoding="utf-8") as f:
                for url in f:
                    #r = urllib.request.urlopen(url)
                    #source = r.read()
                    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
                    webpage = urlopen(req).read()
                    soup = BeautifulSoup(webpage, 'lxml')
                    if name == "bbc":
                        storyDiv = soup.find('div', class_='story-body__inner')
                        storyLis = storyDiv.find_all('p')
                        st = ""
                        for x in storyLis:
                            st = st + str(x.get_text().replace('\n', '').replace('\r', ''))
                        fin = [st]
                        writer.writerow(fin)
                    elif name == "nyt":
                        matches = soup.findAll(classType("story-body-text story-content"))
                        st = ""
                        for sect in matches:
                            st = st + str(sect.get_text())
                        fin = [st]
                        writer.writerow(fin)
                    elif name == "an":
                        storyDiv = soup.find('div', class_='post-content clearfix')
                        sup = storyDiv.find_all('p')
                        st = ""
                        for x in sup:
                            st = st + str(x.get_text().replace('\n', '').replace('\r', ''))
                        finish = [st]
                        writer.writerow(finish)
                    else:
                        storyDiv = soup.find('div', class_='entry-content')
                        storyLis = storyDiv.find_all('p')
                        st = ""
                        for x in storyLis:
                            st = st + str(x.get_text().replace('\n', '').replace('\r', ''))
                        st = re.sub(r'https?:\/\/.*[\r\n]*', '', st)
                        st = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', st)
                        st = re.sub(r"[^'’0-9a-zA-Z#@, ]+", ' ', st)
                        finish = [st]
                        writer.writerow(finish)

In [26]:
def rt():
    with open("rt.csv", 'wb') as result:
        writer = csv.writer(result, dialect='excel')
        with open("rt.txt", 'r', encoding="utf-8") as f:
            for url in f:
                r = urllib.request.urlopen(url).read()
                soup = BeautifulSoup(r, 'lxml')
                storyDiv = soup.find('div', class_='article__text text js-mediator-article')
                storyLis = storyDiv.find_all('p')
                st = ""
                for x in storyLis:
                    st = st + str(x.get_text())
                finish = [st]
                writer.writerow(finish)

In [43]:
rt()

In [31]:
Scrape('an')

In [55]:
#Search out tags for article content
def classType(name):
    #BS4 defines class tags as lists not strings
    name = name.split()
    def matches(minestrone):
        classes = minestrone.get('class', '')
        return all(c in classes for c in name)
    return matches

In [4]:
#Scrape our New York Times articles
def nytScrape():
    with open("nyt.csv", 'wb') as result:
        writer = csv.writer(result, dialect='excel')
        with open(fpath_nyt, 'r', encoding="utf-8") as f:
            for url in f:
                r = urllib.request.urlopen(url).read()
                soup = BeautifulSoup(r, 'lxml')
                #matches = soup.findAll(class_="story-body-text story-content")
                matches = soup.findAll(classType("story-body-text story-content"))
                st = ""
                for sect in matches:
                    st = st + str(sect.get_text())
                fin = [st]
                writer.writerow(fin)

In [5]:
#Scrape our Wall Street Journal articles
def wsjScrape():
    with open("wsj.csv", 'wb') as result:
        writer = csv.writer(result, dialect='excel')
        driver = webdriver.Firefox()
        with open(fpath_wsj, 'r', encoding="utf-8") as f:
            for url in f:
                encoded = urllib.parse.quote(url.encode("utf-8"))
                fullEncode = "https://m.facebook.com/l.php?u=" + encoded
                
                driver.get(fullEncode)
                driver.find_element_by_xpath("//a[@class='_42ft _42fu selected _42g-']").click()
                source = driver.page_source
                soup = BeautifulSoup(source, 'lxml')
                story = soup.findAll('p')
                st = ""
                for x in story[:-14]:
                    st = st + str(x.get_text().replace('\n', '').replace('\r', ''))
                fin = [st]
                writer.writerow(fin)
        driver.close()

In [6]:
def bbcScrape():
    with open("bbc.csv", 'wb') as result:
        writer = csv.writer(result, dialect='excel')
        with open(fpath_bbc, 'r', encoding="utf-8") as f:
            for url in f:
                r = urllib.request.urlopen(url).read()
                soup = BeautifulSoup(r, 'lxml')
                storyDiv = soup.find('div', class_='story-body__inner')
                storyLis = storyDiv.find_all('p')
                st = ""
                for x in storyLis:
                    st = st + str(x.get_text().replace('\n', '').replace('\r', ''))
                fin = [st]
                writer.writerow(fin)

In [16]:
def breitbartScrape():
    with open("bb.csv", 'wb') as result:
        writer = csv.writer(result, dialect='excel')
        with open(fpath_bb, 'r', encoding="utf-8") as f:
            for url in f:
                r = urllib.request.urlopen(url).read()
                soup = BeautifulSoup(r, 'lxml')
                storyDiv = soup.find('div', class_='entry-content')
                storyLis = storyDiv.find_all('p')
                st = ""
                for x in storyLis:
                    st = st + str(x.get_text().replace('\n', '').replace('\r', ''))
                
                st = re.sub(r'https?:\/\/.*[\r\n]*', '', st)
                st = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', st)
                st = re.sub(r"[^'’0-9a-zA-Z#@, ]+", ' ', st)
                finish = [st]
                writer.writerow(finish)
    

In [13]:
def onionScrape():
    with open("theonionlinks.csv", 'wb') as result:
        writer = csv.writer(result, dialect='excel')
        with open(fpath_onion, 'r', encoding="utf-8") as f:
            for url in f:
                r = urllib.request.urlopen(url).read()
                soup = BeautifulSoup(r, 'lxml')
                matches = soup.findAll(class_="content-main-body")
                text = ""
                for x in matches:
                    text = text + str(x.get_text()).replace('\n', '').replace('\r', '')
                finish = [text]
                print(finish)
                writer.writerow(finish)
    result.close()

In [17]:
breitbartScrape()

In [18]:
with open("bb.csv", 'r', encoding="utf-8") as f:
    counter = 0
    for x in f:
        if counter == 0:
            print(x)
            break
        counter = counter + 1

" Bob Corker, who helped President O give us the bad Iran Deal   couldn’t get elected dog catcher in Tennessee, is now fighting Tax Cuts,  Trump wrote on Twitter Bob Corker, who helped President O give us the bad Iran Deal   couldn't get elected dog catcher in Tennessee, is now fighting Tax Cuts  Donald J  Trump  @realDonaldTrump  October 24, 2017 Trump has previously used the  dog catcher  attack on Hillary Clinton during the presidential race, on Sen  Marco Rubio during the Republican presidential primary, and on Mayor Michael Bloomberg of New York City He added that Corker decided not to run for re election after Trump refused to endorse him and had since become opposed to his proposals Bob Corker, who helped President O give us the bad Iran Deal   couldn't get elected dog catcher in Tennessee, is now fighting Tax Cuts  Donald J  Trump  @realDonaldTrump  October 24, 2017 Corker dropped out of the race in Tennesse  sic  when I refused to endorse him, and now is only negative on anyth